# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [3]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [classes[i]] * len(data)
    return data, target

In [4]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i in range(len(classes)):
    data, target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)
    
all_data = np.asarray(all_data)
all_target = np.asarray(all_target)

print 'total number of data', len(all_data)

total number of data 222


In [5]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [6]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [7]:
# YOUR CODE HERE
clf.fit(all_data[training_data], all_target[training_data])
# clf.fit(digits.data[:-500], digits.target[:-500]) 

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### predicting

In [8]:
clf.predict(all_data[-1]), all_target[-1]

ValueError: Expected 2D array, got 1D array instead:
array=[-0.24693203  0.12582994 -0.28681612  0.99552417 -0.24693203 -0.12421203
 -0.30049455  1.01094818  0.01514834  0.11869175].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [9]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [10]:
expected = []
predicted = []
# YOUR CODE HERE
expected = all_target[training_data]
predicted = clf.predict(all_data[training_data])
evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

        Back       1.00      1.00      1.00        16
       Belly       1.00      1.00      1.00        11
      Crouch       0.96      1.00      0.98        27
        Frog       1.00      1.00      1.00         4
    HeadBack       1.00      1.00      1.00         9
        Knee       1.00      1.00      1.00         7
        Left       1.00      1.00      1.00        14
       Right       1.00      0.86      0.92         7
         Sit       1.00      1.00      1.00        17
       Stand       1.00      1.00      1.00         7
   StandInit       1.00      1.00      1.00        36

   micro avg       0.99      0.99      0.99       155
   macro avg       1.00      0.99      0.99       155
weighted avg       0.99      0.99      0.99       155


Confusion matrix:
[[16  0  0  0  0  0  0  0  0  0  0]
 [ 0 11  0  0  0  0  0  0  0  0  0]
 [ 0  0 27  0  0  0  0  0  0  0  0]
 [ 0  0  0  4  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [11]:
expected = []
predicted = []
# YOUR CODE HERE
test_data = permutation[n_training_data:]
expected = all_target[test_data]
predicted = clf.predict(all_data[test_data])
evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

        Back       1.00      1.00      1.00         7
       Belly       1.00      1.00      1.00         8
      Crouch       1.00      1.00      1.00         3
        Frog       1.00      1.00      1.00         6
    HeadBack       1.00      1.00      1.00         1
        Knee       1.00      1.00      1.00         3
        Left       1.00      1.00      1.00         6
       Right       1.00      1.00      1.00         4
         Sit       1.00      1.00      1.00         9
       Stand       1.00      1.00      1.00         4
   StandInit       1.00      1.00      1.00        16

   micro avg       1.00      1.00      1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67


Confusion matrix:
[[ 7  0  0  0  0  0  0  0  0  0  0]
 [ 0  8  0  0  0  0  0  0  0  0  0]
 [ 0  0  3  0  0  0  0  0  0  0  0]
 [ 0  0  0  6  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [12]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [13]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1]), all_target[-1]

ValueError: Expected 2D array, got 1D array instead:
array=[-0.24693203  0.12582994 -0.28681612  0.99552417 -0.24693203 -0.12421203
 -0.30049455  1.01094818  0.01514834  0.11869175].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.